In [1]:
!wget https://raw.githubusercontent.com/marybarker/gerrymandering/master/NorthCarolina/adjacencyGeoms.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
# from geopy.distance import vincenty as latlongdist
import geopy.distance
from helper import *
import ast
import random

#Read date
bounds_raw = pd.read_csv('adjacencyGeoms.csv',index_col=None)
bounds_raw = bounds_raw#.iloc[:50,:]
idx = bounds_raw['geom'].apply(len)>2
bounds_raw = bounds_raw.loc[idx]

In [7]:
bounds_raw.head()

,low,high,geom
0,37149WO06,37149SA04,"[[(-82.243803, 35.266116, 0.0), (-82.24381, 35..."
2,37149WO06,37149CL07,"[[(-82.078988, 35.286422, 0.0), (-82.079599, 3..."
3,37149WO06,37149GC09,"[[(-82.04003999999999, 35.316348, 0.0), (-82.0..."
4,37149WO06,37149CG05,"[[(-82.27895, 35.301348, 0.0), (-82.278784, 35..."
5,37149WO06,3716134,"[[(-82.040438, 35.31718, 0.0), (-82.040328, 35..."


In [ ]:
# Process data
def vtd_idx(row):
    low_idx = inv_vtds[row['low']]
    high_idx = inv_vtds[row['high']]
    if low_idx > high_idx:
        low_idx, high_idx = high_idx, low_idx

    pts = np.array(ast.literal_eval(row['geom'])[0])    
    pts = pts[:,:2]    
    pairs = zip(pts,pts[1:])
    length = 0
    for (xi,xf) in pairs:
        # geopy.distance.vincenty is a more accurate but slow way to convert lat-long difference into distance
#         dl = geopy.distance.vincenty(xi,xf).meters

        # this is standard Euclidean distance - fast but not as accurate
        dx = xf-xi
        dl = np.sqrt(dx.dot(dx))
        length += dl
    return [low_idx, high_idx, (low_idx,high_idx), length, vtd_geoids[low_idx], vtd_geoids[high_idx], pts]

vtd_geoids = np.unique(bounds_raw[['low','high']]).tolist()
num_vtds = len(vtd_geoids)
inv_vtds = {vtd: i for i, vtd in enumerate(vtd_geoids)}

new = [vtd_idx(row[1]) for row in bounds_raw.iterrows()]

bounds = pd.DataFrame.from_records(new,columns=['low_idx','high_idx','pair','length','low_geo','high_geo','pts'])
bounds.sort_values('pair',inplace=True)
bounds.reset_index(inplace=True)
# bounds.set_index('pair',inplace=True)

pop=np.random.randint(100000,size=num_vtds)

In [17]:
bounds.head()

,index,low_idx,high_idx,pair,length,low_geo,high_geo,pts
0,4158,0,1,"(0, 1)",0.108070,3700101,3700102,"[[-79.43251, 35.960426], [-79.432729, 35.96043..."
1,4174,0,15,"(0, 15)",0.010172,3700101,3700107,"[[-79.43251, 35.960426], [-79.43258, 35.95959]..."
2,4173,0,16,"(0, 16)",0.061166,3700101,3700108N,"[[-79.432659, 35.950257], [-79.432701, 35.9471..."
3,4172,0,17,"(0, 17)",0.045365,3700101,3700108S,"[[-79.431972, 35.889104], [-79.431975, 35.8890..."
4,4175,0,439,"(0, 439)",0.109713,3700101,370373,"[[-79.432778, 35.843749], [-79.435062, 35.8437..."


In [9]:
G = nx.Graph()
for (i,vtd) in enumerate(vtd_geoids):
    G.add_node(i, geoid=vtd, pop=pop[i])
for (b, attr) in bounds.iterrows():
    if attr['pair'] in G.edges:
        G.edges[attr['pair']]['length'] += attr['length']
    else:
        G.add_edge(*attr['pair'], length=attr['length'])

In [19]:
G.edges()

EdgeView([(0, 1), (0, 15), (0, 16), (0, 17), (0, 439), (0, 1077), (0, 2003), (1, 5), (1, 10), (1, 15), (1, 30), (1, 1077), (1, 1132), (2, 3), (2, 4), (2, 6), (2, 7), (2, 25), (2, 28), (2, 1145), (3, 4), (3, 5), (3, 6), (3, 25), (3, 27), (4, 6), (5, 6), (5, 26), (5, 27), (5, 30), (5, 1132), (6, 1076), (6, 1145), (7, 8), (7, 28), (7, 391), (7, 1126), (7, 1145), (7, 2095), (8, 9), (8, 23), (8, 28), (8, 35), (8, 383), (8, 391), (9, 12), (9, 28), (9, 31), (9, 32), (9, 35), (10, 11), (10, 13), (10, 14), (10, 15), (10, 30), (11, 12), (11, 13), (11, 14), (11, 35), (12, 14), (12, 31), (12, 35), (13, 15), (13, 18), (13, 20), (13, 35), (14, 24), (14, 31), (14, 33), (15, 16), (15, 18), (15, 19), (16, 17), (16, 19), (16, 1870), (17, 437), (17, 439), (17, 449), (17, 1870), (18, 19), (18, 20), (19, 20), (19, 1843), (19, 1862), (19, 1870), (20, 22), (20, 35), (20, 1843), (21, 22), (21, 23), (21, 35), (21, 1843), (22, 35), (22, 1843), (23, 35), (23, 383), (23, 389), (23, 1833), (24, 26), (24, 29), (24,

In [20]:
# initial district assignment
num_dists = 3

if num_dists > num_vtds:
    raise Exception('More districts than vtds', num_dist, num_vtds)

vtds_left = set(range(num_vtds))
dist_vtds = [[] for _ in range(num_dists)]  #list of lists of vtds in each district
vtd_dist = np.empty(num_vtds).astype(int)  #list of dist assigned to each vtd
dist_pop = np.zeros(num_dists).astype(int)

def update(dist, vtds):
    global dist_vtds, vtds_left, dist_pop, G
    dist_vtds[dist].extend(vtds)  # put new vtds in the dist list
    vtds_left.difference_update(vtds)  # remove from unassigned vtds
    for v in vtds:
        vtd_dist[v] = dist  
        G.nodes[v]['dist'] = dist  # record in graph object        
        dist_pop[dist] += pop[v]  # increase dist population

# First vtd for each district
for dist in range(num_dists):
    v = random.sample(vtds_left, 1)
    update(dist, v)
    
# grow districts
while bool(vtds_left):
    done = False
    for dist in dist_pop.argsort():  # Add to district with smallest population if possible
        vtds_in = dist_vtds[dist].copy()
        random.shuffle(vtds_in)  # Grow from a random vtd in the dist
        for v in vtds_in:
            neighbors_left = set(G.neighbors(v)).intersection(vtds_left)  # find avaiable neighboring vtds
            if bool(neighbors_left):  # if some, great.  Add all of them to the district and turn the while loop.  Else, try again
                update(dist, neighbors_left)
                done = True
                break
        if done == True:
            break
    if done == False:
        raise Exception("No place to put %s" % vtds_left)
        
bounds['low_dist'] = vtd_dist[bounds['low_idx']]
bounds['high_dist'] = vtd_dist[bounds['high_idx']]

In [4]:
# part of a step of the markov chain
# to add - does this make the map better or worse
# I have not coded any goodness calculations yet

cross = bounds['low_dist'] != bounds['high_dist']
e = random.choice(cross.index)
if random.getrandbits(1):
    vtd_move = bounds.loc[e,'low_idx']
    dist_new = bounds.loc[e,'high_dist']    
else:
    vtd_move = bounds.loc[e,'high_idx']
    dist_new = bounds.loc[e,'low_dist']

dist_left = vtd_dist[vtd_move]
dist_vtds[dist_left].remove(vtd_move)
dist_vtds[dist_new].append(vtd_move)
vtd_dist[vtd_move] = dist_new

pop[dist_left] -= pop[vtd_move]
pop[dist_new] += pop[vtd_move]
bounds['low_dist'] = vtd_dist[bounds['low_idx']]
bounds['high_dist'] = vtd_dist[bounds['high_idx']]


# end of program so far

In [7]:
print(vtds_left)
print(dist_pop)
for (vtds,pop) in zip(dist_vtds,dist_pop):
    print(pop)
    print(vtds)


set()

[33562511 41427441 61964635]
33562511
[1808, 1819, 1818, 1827, 1824, 1826, 1828, 1829, 1830, 1809, 1811, 1812, 1813, 1816, 1817, 1822, 1912, 1823, 1810, 699, 356, 1821, 374, 359, 696, 1825, 698, 1814, 1909, 1815, 1820, 362, 355, 351, 704, 709, 1370, 379, 1914, 1911, 1902, 1903, 1905, 1907, 1908, 1368, 1366, 1896, 1906, 1900, 1895, 697, 706, 707, 1752, 1913, 1901, 1765, 1904, 1910, 1767, 1789, 1766, 1760, 1763, 369, 371, 380, 368, 370, 364, 361, 554, 549, 1367, 375, 702, 694, 1761, 1771, 366, 367, 1881, 550, 551, 372, 365, 382, 1369, 1365, 703, 349, 353, 354, 539, 1364, 540, 1397, 1390, 1391, 1768, 1762, 1779, 360, 373, 352, 1876, 381, 358, 708, 1394, 693, 1785, 1781, 1392, 1393, 541, 1775, 545, 546, 547, 536, 538, 542, 1385, 1386, 1379, 1389, 1952, 1942, 110, 1954, 1948, 1949, 1950, 1376, 1947, 1937, 1938, 544, 109, 543, 1395, 1384, 1396, 982, 983, 1377, 1381, 1875, 111, 120, 106, 1874, 552, 560, 548, 556, 1380, 1383, 553, 555, 557, 535, 2601, 1388, 561, 1880, 1877, 1878, 1879, 

In [4]:
for (node, attr) in G.nodes.items():
    print(node, attr)

for (edge, attr) in G.edges.items():
    print(edge, attr)

0 {'geoid': '37089RR', 'pop': 15069}
1 {'geoid': '37089SB', 'pop': 89638}
2 {'geoid': '37149CG05', 'pop': 41864}
3 {'geoid': '37149CL07', 'pop': 86245}
4 {'geoid': '37149CL08', 'pop': 45031}
5 {'geoid': '37149GC09', 'pop': 40345}
6 {'geoid': '37149SA04', 'pop': 89519}
7 {'geoid': '37149TR123', 'pop': 24119}
8 {'geoid': '37149WO06', 'pop': 39933}
9 {'geoid': '3716109A', 'pop': 22206}
10 {'geoid': '3716118', 'pop': 50996}
11 {'geoid': '3716134', 'pop': 53981}
(0, 6) {'length': 16526.454417419558}
(1, 2) {'length': 808.2193120401539}
(1, 6) {'length': 17982.9591967458}
(2, 6) {'length': 12617.61512366936}
(2, 8) {'length': 150425.62888084815}
(3, 4) {'length': 55238.95208649865}
(3, 5) {'length': 4494.0125904316565}
(3, 7) {'length': 7057.0783938471295}
(3, 8) {'length': 60668.20263764371}
(4, 5) {'length': 45749.71942241032}
(4, 7) {'length': 10045.356911464747}
(5, 8) {'length': 14683.798432782403}
(5, 9) {'length': 7538.698468944758}
(5, 11) {'length': 30320.67234689818}
(6, 7) {'lengt